In [20]:
import pandas as pd
from rdkit.Chem import PandasTools, AllChem
from rdkit import DataStructs
from rdkit import Chem
import numpy as np

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_validate, RandomizedSearchCV, StratifiedKFold
from sklearn import metrics

from imblearn.under_sampling import NearMiss

from statistics import mean
from math import floor

In [44]:
def genMorgan(mol):
    funcFPInfo=dict(radius=3,nBits=2048,useFeatures=False,useChirality = False)
    #print(type(mol))
    return AllChem.GetMorganFingerprintAsBitVect(mol,**funcFPInfo)

def convertVector(v):
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(v,arr)
    return arr

def balanceNearMiss(df,near_miss_strat):
    bigger_set = df.loc[df.Outcome == 1]
    smaller_set = df.loc[df.Outcome == 0]
    if len(bigger_set) < len(smaller_set):
        bigger_set = df.loc[df.Outcome == 0]
        smaller_set = df.loc[df.Outcome == 1]
    if len(smaller_set)/len(bigger_set) < .9:
        X = pd.DataFrame(df['maccs'].apply(convertVector).tolist())
        sampler = NearMiss(sampling_strategy='majority',version=near_miss_strat)
        fitted = sampler.fit_resample(X,df.Outcome)
        sample_idx = sampler.sample_indices_
        return df.iloc[sample_idx,:].copy()
    else:
        return df
                            
def stats(y_train, y_pred):
    confusion_matrix = metrics.confusion_matrix(y_train, y_pred)
    print(confusion_matrix)
    accuracy = metrics.accuracy_score(y_train, y_pred)
    roc_auc_score = metrics.roc_auc_score(y_train, y_pred)
    Kappa = metrics.cohen_kappa_score(y_train, y_pred, weights='linear')
    # True and false values
    TN, FP, FN, TP = confusion_matrix.ravel()  
    # Sensitivity, hit rate, recall, or true positive rate
    SE = TP/(TP+FN)
    # Specificity or true negative rate
    SP = TN/(TN+FP) 
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Correct classification rate
    CCR = (SE + SP)/2
    d = dict({'Accuracy': accuracy,
         'AUC': roc_auc_score,
         'Kappa': Kappa,
         'CCR': CCR,
         'Sensitivity': SE,
         'PPV': PPV,
         'Specificity': SP,
         'NPV': NPV})
    return pd.DataFrame(d, columns=d.keys(), index=[0]).round(2)

# Load Data and Binarize Outcome

In [4]:
df = PandasTools.LoadSDF("/home/francis/Documents/MML/StopTox/Data/1-Acute_dermal_2616_st.sdf")
df['Outcome'] = df['LD50'].map(lambda x: 1 if float(x) > 2000 else 0)

# Generate MACCS Fingerprints

In [5]:
df['maccs'] = df['ROMol'].apply(Chem.rdMolDescriptors.GetMACCSKeysFingerprint)

# Balance Using NearMiss

In [6]:
df_balanced = balanceNearMiss(df,2)
# df_balanced = balanceNearMiss(df,1)

In [7]:
training_maccs = np.array(list(df_balanced['maccs'])).astype(int)

# Hyperparameter tuning

In [8]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [51]:
model = RFC(n_estimators=1000,
            criterion='entropy',
            max_features='sqrt',
            max_depth=None,
            min_samples_split=5,
            min_samples_leaf=1,
            bootstrap=True,
            random_state=24,
            verbose=1,
            n_jobs=-1)
rf_random = RandomizedSearchCV(estimator = model,
                               param_distributions = random_grid,
                               n_iter = 100, cv = 5,
                               verbose=2,
                               random_state=24,
                               n_jobs = -1)

In [12]:
%%time
rf_random.fit(training_maccs,df_balanced.Outcome)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   44.0s
/home/francis/Programs/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 10.0min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.8s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    1.4s
[Parallel(n_jobs=2)]: Done 

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0,
            n_estimators=[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
            n_jobs=2, oob_score=False, random_state=24, verbose=1,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=24, refit=True,
        

In [39]:
params = rf_random.best_params_
chosen_maccs_model = RFC(n_estimators=params['n_estimators'],
            criterion='entropy',
            max_features=params['max_features'],
            max_depth=params['max_depth'],
            min_samples_split=params['min_samples_split'],
            min_samples_leaf=params['min_samples_leaf'],
            bootstrap=params['bootstrap'],
            random_state=24,
            verbose=1,
            n_jobs=-1)

In [40]:
df_balanced['maccs_pred'] = [0 for _ in range(len(df_balanced))]
pred_pos = df_balanced.columns.get_loc('maccs_pred')
outcome_pos = df_balanced.columns.get_loc('Outcome')

In [41]:
%%time
skf = StratifiedKFold(5)
for train_idx, test_idx in skf.split(training_maccs,df_balanced.Outcome):
    #print(training_maccs[train_idx],df.Outcome[train_idx])
    chosen_maccs_model.fit(training_maccs[train_idx],df_balanced.iloc[train_idx,outcome_pos])
    df_balanced.iloc[test_idx,pred_pos] = chosen_maccs_model.predict(training_maccs[test_idx])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 1600 out of 1600 | elapsed:    1.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 1600 out of 1600 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent worke

CPU times: user 31.6 s, sys: 1.74 s, total: 33.4 s
Wall time: 16 s


[Parallel(n_jobs=4)]: Done 1600 out of 1600 | elapsed:    0.2s finished


In [45]:
maccs_result = stats(df_balanced.Outcome,df_balanced.maccs_pred)

[[472 219]
 [223 468]]


In [46]:
maccs_result

,Accuracy,AUC,Kappa,CCR,Sensitivity,PPV,Specificity,NPV
0,0.68,0.68,0.36,0.68,0.68,0.68,0.68,0.68


In [49]:
df_balanced['morgan'] = df_balanced['ROMol'].apply(lambda x: genMorgan(x))
training_morgan = np.array(list(df_balanced['morgan'])).astype(int)

In [50]:
%%time
rf_random.fit(training_morgan,df_balanced.Outcome)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.8min
/home/francis/Programs/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: 

In [ ]:
params = rf_random.best_params_
chosen_morgan_model = RFC(n_estimators=params['n_estimators'],
            criterion='entropy',
            max_features=params['max_features'],
            max_depth=params['max_depth'],
            min_samples_split=params['min_samples_split'],
            min_samples_leaf=params['min_samples_leaf'],
            bootstrap=params['bootstrap'],
            random_state=24,
            verbose=1,
            n_jobs=-1)

In [ ]:
df_balanced['morgan_pred'] = [0 for _ in range(len(df_balanced))]
pred_pos = df_balanced.columns.get_loc('morgan_pred')

In [ ]:
%%time
skf = StratifiedKFold(5)
for train_idx, test_idx in skf.split(training_morgan,df_balanced.Outcome):
    #print(training_maccs[train_idx],df.Outcome[train_idx])
    chosen_morgan_model.fit(training_morgan[train_idx],df_balanced.iloc[train_idx,outcome_pos])
    df_balanced.iloc[test_idx,pred_pos] = chosen_maccs_model.predict(training_morgan[test_idx])